In [1]:
import pandas as pd
import requests
import mysql.connector

In [3]:
API_KEY = ''
CHANNEL_ID = 'UC2UXDak6o7rBm23k3Vv5dww'
PAGE_TOKEN = ''

searchUrl = 'https://www.googleapis.com/youtube/v3/search?key=' + API_KEY + '&part=snippet' + '&type=video' + '&channelId=UCG5qGWdu8nIRZqJ_GgDwQ-w' + '&order=date' + '&maxResults=50' + '&pageToken='
videosUrl = 'https://www.googleapis.com/youtube/v3/videos?key=' + API_KEY + '&part=snippet,contentDetails,statistics' + '&id='

df = pd.DataFrame(columns=['videoId', 'publishedAt', 'title', 'tags', 'duration', 'viewCount', 'likeCount', 'commentCount'])

In [4]:
while True:
    try:
        searchResponse = requests.get(searchUrl + PAGE_TOKEN).json()
        PAGE_TOKEN = searchResponse['nextPageToken']
        print(len(df.index))

        for video in searchResponse['items']:
            videoId = video['id']['videoId']
            videoResponse = requests.get(videosUrl + videoId).json()
            
            publishedAt = videoResponse['items'][0]['snippet']['publishedAt']
            title = videoResponse['items'][0]['snippet']['title']
            tags = videoResponse['items'][0]['snippet']['tags']

            duration = videoResponse['items'][0]['contentDetails']['duration']

            viewCount = videoResponse['items'][0]['statistics']['viewCount']
            likeCount = videoResponse['items'][0]['statistics']['likeCount']
            commentCount = videoResponse['items'][0]['statistics']['commentCount']

            data = {'videoId' : videoId, 
                    'publishedAt' : publishedAt, 
                    'title' : title, 
                    'tags' : tags,
                    'duration' : duration,
                    'viewCount' : viewCount,
                    'likeCount' : likeCount,
                    'commentCount' : commentCount}
            df = pd.concat([df, pd.Series(data).to_frame().T], ignore_index=True)
            
    except:
        print(len(df.index))
        break

0
50
100
150
200
250
300
350
400
450
500


In [7]:
df

#df.to_csv("videos.csv")

,videoId,publishedAt,title,tags,duration,viewCount,likeCount,commentCount
0,20_wpq65YB4,2023-06-29T15:01:15Z,James Maddison MASTERCLASS vs Nottingham Fores...,"[James Maddison, Maddison, masterclass, Totten...",PT6M2S,27837,949,70
1,2nimbJKUqAE,2023-06-29T09:31:36Z,Eberechi Eze with a SENSATIONAL solo goal!,"[Eberechi Eze, Eze, solo goal, solo goal footb...",PT12S,197633,7495,49
2,x3juwrll0n8,2023-06-28T20:04:48Z,James Maddison first time finish vs Spurs,"[James Maddison, Maddison, Spurs, Tottenham, T...",PT28S,887069,30172,79
3,7O3FzXkm0UY,2023-06-28T19:09:21Z,Arsenal GK denies Kai Havertz ❗️ #shorts,"[Arsenal, Kai Havertz, Chelsea, Havertz, Kai H...",PT24S,537480,16988,134
4,Cj78dlDLgdE,2023-06-28T17:19:57Z,STUNNING Edouard Mendy save! #shorts,"[Edouard Mendy, Edouard Mendy save, Edouard Me...",PT8S,202941,7143,40
...,...,...,...,...,...,...,...,...
495,1hRc7y7UIWI,2021-06-30T15:10:46Z,FIRST & LAST Premier League goals against the ...,"[#PremierLeague, #FirstGoal, #LastGoal, #Goals...",PT8M33S,267709,4536,160
496,Ot1O97maPps,2021-06-14T10:35:17Z,Premier League Stars at Euro 2020 | Group C | ...,"[Euro 2020, European Championship, Premier Lea...",PT4M31S,31105,793,107
497,obRRsVlPWHc,2021-06-12T07:39:00Z,Premier League Stars at Euro 2020 | Group B | ...,"[Euro 2020, Euros, Premier League, EPL, Soccer...",PT13M3S,65625,1671,149
498,7Rku9RCq6TQ,2021-06-11T18:43:17Z,Premier League Stars at Euro 2020 | Group A | ...,"[Euro 2020, Euros, Premier League, EPL, PL, So...",PT7M9S,225622,2762,221


In [13]:
mydb = mysql.connector.connect(
  host='localhost',
  user='root',
  password='Test1234',
  database ='dfdb'
)
print(mydb)
mycursor = mydb.cursor()

In [14]:
#mycursor.execute('CREATE DATABASE dfdb')

#mycursor.execute('SHOW DATABASES')
#for db in mycursor:
#    print(db)

#mycursor.execute("""
#CREATE TABLE IF NOT EXISTS original (
#  video_id VARCHAR(255) PRIMARY KEY,
#  upload_date TEXT,
#  video_title TEXT,
#  video_tags TEXT,
#  video_duration TEXT,
#  video_view INTEGER,
#  video_like INTEGER,
#  video_comment INTEGER
#)""")

In [15]:
checkFormula = 'SELECT video_id FROM original WHERE video_id=%s'
updateForumla = 'UPDATE original SET video_view = %s, video_like = %s, video_comment = %s WHERE video_id = %s;'
insertForumla = 'INSERT INTO original (video_id, upload_date, video_title, video_tags, video_duration, video_view, video_like, video_comment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);'

for i, row in df.iterrows():
    mycursor.execute(checkFormula, (row['videoId'], ))
    if mycursor.fetchone() is not None: #check if it exists
        #update
        rowData = (
            row['viewCount'],
            row['likeCount'],
            row['commentCount'],
            row['videoId']
        )
        mycursor.execute(updateForumla, rowData)
    else:
        #insert
        rowData = (
            row['videoId'],
            row['publishedAt'],
            row['title'],
            ', '.join(row['tags']),
            row['duration'],
            row['viewCount'],
            row['likeCount'],
            row['commentCount']
        )
        mycursor.execute(insertForumla, rowData)

mydb.commit()